In [30]:
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from molfeat.trans.fp import FPVecTransformer

In [31]:
df_data = pd.read_csv('EGFR_Feature_Extraction.csv')
df_data.head()

,molecule_chembl_id,canonical_smiles,Molecular Weight,Number of Rotatable Bonds,Number of Atoms,Number of Bonds,Count of Chiral Centers,Number of Rings,Number of Aromatic Rings,Number of Hydrogen Bond Donors,Number of Hydrogen Bond Acceptors,Balaban J Index,Wiener Index,LogP,TPSA,standard_value,class
0,CHEMBL68920,Cc1cc(C)c(/C=C2\C(=O)Nc3ncnc(Nc4ccc(F)c(Cl)c4)...,383.094916,3,27,30,0,4,3,3,4,1.789174,1832.0,4.45034,82.70,41.0,active
1,CHEMBL137635,CN(c1ccccc1)c1ncnc2ccc(N/N=N/Cc3ccccn3)cc12,369.170194,6,28,31,0,4,4,1,6,1.558763,2326.0,4.77200,78.66,9300.0,inactive
2,CHEMBL306988,CC(=C(C#N)C#N)c1ccc(NC(=O)CCC(=O)O)cc1,283.095691,5,21,21,0,1,1,2,4,2.782020,1108.0,2.31056,113.98,500000.0,inactive
3,CHEMBL66879,O=C(O)/C=C/c1ccc(O)cc1,164.047344,2,12,12,0,1,1,2,2,2.772026,223.0,1.49000,57.53,3000000.0,inactive
4,CHEMBL77085,N#CC(C#N)=Cc1cc(O)ccc1[N+](=O)[O-],215.033091,2,16,16,0,1,1,1,5,3.386653,441.0,1.73096,110.95,96000.0,inactive


In [32]:
df = df_data.drop(df_data.columns[-2], axis=1, inplace=True)
df = df_data.iloc[:, 2:]
print(df['class'].value_counts())
df.head()

class
active      3810
inactive    3524
Name: count, dtype: int64


,Molecular Weight,Number of Rotatable Bonds,Number of Atoms,Number of Bonds,Count of Chiral Centers,Number of Rings,Number of Aromatic Rings,Number of Hydrogen Bond Donors,Number of Hydrogen Bond Acceptors,Balaban J Index,Wiener Index,LogP,TPSA,class
0,383.094916,3,27,30,0,4,3,3,4,1.789174,1832.0,4.45034,82.70,active
1,369.170194,6,28,31,0,4,4,1,6,1.558763,2326.0,4.77200,78.66,inactive
2,283.095691,5,21,21,0,1,1,2,4,2.782020,1108.0,2.31056,113.98,inactive
3,164.047344,2,12,12,0,1,1,2,2,2.772026,223.0,1.49000,57.53,inactive
4,215.033091,2,16,16,0,1,1,1,5,3.386653,441.0,1.73096,110.95,inactive


In [33]:
X = df.drop('class', axis=1)
y = df['class']

In [34]:
smiles = df_data['canonical_smiles']
transformer = FPVecTransformer(kind='desc2D', dtype=float)
features = transformer(smiles)
features_df = pd.DataFrame(features, index=smiles.index)  # Convert to DataFrame
X = pd.concat([X, features_df], axis=1)
X.head()


/home/jjk339/miniconda3/envs/FirstProject/lib/python3.12/site-packages/molfeat/calc/descriptors.py:46: RuntimeWarning: All-NaN slice encountered
  min_charge, max_charge = np.nanmin(atomic_charges), np.nanmax(atomic_charges)


,Molecular Weight,Number of Rotatable Bonds,Number of Atoms,Number of Bonds,Count of Chiral Centers,Number of Rings,Number of Aromatic Rings,Number of Hydrogen Bond Donors,Number of Hydrogen Bond Acceptors,Balaban J Index,Wiener Index,LogP,TPSA,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,...,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215
0,383.094916,3,27,30,0,4,3,3,4,1.789174,1832.0,4.45034,82.70,13.399867,13.399867,0.006339,-0.509118,0.583357,14.370370,383.814,368.694,383.094916,136.0,0.0,0.257300,-0.358918,0.358918,0.257300,1.222222,2.037037,2.740741,35.495692,10.072045,2.293219,-2.097428,2.370823,-2.039343,6.353190,-0.110341,3.105192,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,369.170194,6,28,31,0,4,4,1,6,1.558763,2326.0,4.77200,78.66,4.485145,4.485145,0.423142,0.423142,0.391062,11.035714,369.432,350.280,369.170194,138.0,0.0,0.191859,-0.328866,0.328866,0.191859,0.928571,1.785714,2.607143,15.413135,10.164663,2.105032,-2.194233,2.291848,-2.114184,5.929560,0.889676,2.768899,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2,283.095691,5,21,21,0,1,1,2,4,2.782020,1108.0,2.31056,113.98,11.466694,11.466694,0.035411,-1.026115,0.804599,9.095238,283.287,270.183,283.095691,106.0,0.0,0.303365,-0.481220,0.481220,0.303365,1.142857,1.714286,2.190476,16.365276,10.024847,2.069205,-2.061446,2.196544,-2.125729,5.921818,-0.137870,2.467695,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,164.047344,2,12,12,0,1,1,2,2,2.772026,223.0,1.49000,57.53,10.103372,10.103372,0.168958,-0.983264,0.650834,10.333333,164.160,156.096,164.047344,62.0,0.0,0.327821,-0.507966,0.507966,0.327821,1.250000,1.833333,2.333333,16.364576,10.179853,1.889192,-1.916154,2.098143,-1.846733,5.847442,-0.130858,2.033230,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,215.033091,2,16,16,0,1,1,1,5,3.386653,441.0,1.73096,110.95,10.620054,10.620054,0.016759,-0.655117,0.458317,8.625000,215.168,210.128,215.033091,78.0,0.0,0.293093,-0.507935,0.507935,0.293093,1.437500,2.062500,2.562500,16.628356,10.102218,2.124462,-1.976004,2.247448,-1.910268,5.690520,-0.384628,2.250601,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0


In [35]:
le = LabelEncoder()
y = le.fit_transform(y)

In [36]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Train size: " + str(X_train.shape[0]))
print("Test size: " + str(X_test.shape[0]))
print("Number of features: " + str(X_train.shape[1]))
print("Number of classes: " + str(len(le.classes_)))

Train size: 5867
Test size: 1467
Number of features: 229
Number of classes: 2


In [37]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [38]:
params = {
    'task': 'train',
    'objective': 'binary',
    'boosting_type': 'dart',
    'data_sample_strategy': 'bagging',
    'tree_learner': 'data',
    'metric': 'binary_logloss',
    'num_leaves': 71,
    'learning_rate': 0.3,
    'feature_fraction': 0.9,
}

In [39]:
model = lgb.train(params=params, 
                  train_set=train_data, 
                  num_boost_round=400,
                  valid_sets=[test_data]) 

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2821, number of negative: 3046
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.013231 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24863
[LightGBM] [Info] Number of data points in the train set: 5867, number of used features: 207
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.480825 -> initscore=-0.076738
[LightGBM] [Info] Start training from score -0.076738
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [40]:
y_pred = model.predict(X_test, num_iteration=model.best_iteration)
y_pred_binary = [1 if x > 0.5 else 0 for x in y_pred]
accuracy = accuracy_score(y_test, y_pred_binary)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9154737559645535
